In [ ]:
%pip install html5lib 

In [3]:
import os
import pandas as pd

SCORE_DIR = "/Users/moneysniper/Documents/NBA_analysis_project/data/scores"

In [4]:
box_scores = os.listdir(SCORE_DIR)
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [5]:
from bs4 import BeautifulSoup

def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [6]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [7]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    
    return line_score

In [8]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [9]:
processed_files = set()

try:
    with open('processed_files.txt', 'r') as f:
        processed_files = set(line.strip() for line in f)
except FileNotFoundError:
    pass

games = []
base_cols = None

for box_score in box_scores:
    if box_score in processed_files:
        continue
    try:
        soup = parse_html(box_score)
        line_score = read_line_score(soup)
    except ValueError:
        print(f"Error processing file: {box_score}")
        continue
    teams= list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")
        
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index = totals.index.str.lower()
        
        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"
        
        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
                base_cols = list(summary.index.drop_duplicates(keep="first"))
                base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]
            
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")
        
    processed_files.add(box_score)
    with open('processed_files.txt', 'w') as f:
        for file in processed_files:
            f.write(file + '\n')

100 / 11510
200 / 11510
300 / 11510
400 / 11510
500 / 11510
600 / 11510
700 / 11510
800 / 11510
900 / 11510
1000 / 11510
1100 / 11510
1200 / 11510
1300 / 11510
1400 / 11510
1500 / 11510
1600 / 11510
1700 / 11510
1800 / 11510
1900 / 11510
2000 / 11510
2100 / 11510
2200 / 11510
2300 / 11510
2400 / 11510
2500 / 11510
2600 / 11510
2700 / 11510
2800 / 11510
2900 / 11510
3000 / 11510
3100 / 11510
3200 / 11510
3300 / 11510
3400 / 11510
3500 / 11510
3600 / 11510
3700 / 11510
3800 / 11510
3900 / 11510
4000 / 11510
4100 / 11510
4200 / 11510
4300 / 11510
4400 / 11510
4500 / 11510
4600 / 11510
4700 / 11510
4800 / 11510
4900 / 11510
5000 / 11510
5100 / 11510
5200 / 11510
5300 / 11510
5400 / 11510
5500 / 11510
5600 / 11510
5700 / 11510
5800 / 11510
5900 / 11510
6000 / 11510
6100 / 11510
6200 / 11510
6300 / 11510
6400 / 11510
6500 / 11510
6600 / 11510
6700 / 11510
6800 / 11510
6900 / 11510
7000 / 11510
7100 / 11510
7200 / 11510
7300 / 11510
7400 / 11510
7500 / 11510
7600 / 11510
7700 / 11510
7800 / 1

In [10]:
games_df = pd.concat(games, ignore_index=True)
games_df.to_csv("/Users/moneysniper/Documents/NBA_analysis_project/gamelogs/nba_games.csv")